In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests

# Import API key
from api_keys import api_key

In [3]:
oscar_data= pd.read_csv("oscar_data.csv")
oscar_data.head()

,Year,Host,Most Nominations,Most Wins,Best Picture,Nominee 1,Nominee 2,Nominee 3,Nominee 4,Nominee 5,...,Best Costume Design (Black and White),Nominee 1.31,Nominee 2.31,Nominee 3.30,Nominee 4.30,Best Costume Design (Color),Nominee 1.32,Nominee 2.32,Nominee 3.31,Nominee 4.31
0,1927/28,Douglas Fairbanks,Seventh Heaven (5),Seventh Heaven AND Sunrise: A Song of Two Huma...,Wings,The Racket,Seventh Heaven,Unique and Artistic Production- Sunrise: A Son...,Chang: A Drama of the Wilderness,The Crowd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1928/29,William C. DeMille,In Old Arizona AND The Patriot (5),None (No Film Won More Than One Award),The Broadway Melody,Alibi,Hollywood Revue,In Old Arizona,The Patriot,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1929/30,Conrad Nagel,The Love Parade (6),All Quiet on the Western Front AND The Big Hou...,All Quiet on the Western Front,The Big House,Disraeli,The Divorcee,The Love Parade,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1930/31,Lawrence Grant,Cimarron (7),Cimarron (3),Cimarron,East Lynne,The Front Page,Skippy,Trader Horn,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1931/32,Conrad Nagel,Arrowsmith AND The Champ (4),Bad Girl AND The Champ (2),Grand Hotel,Arrowsmith,Bad Girl,The Champ,Five Star Final,One Hour with You,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
nominees = oscar_data[["Best Picture", "Nominee 1", "Nominee 2", "Nominee 3", "Nominee 4"]]
nominees

,Best Picture,Nominee 1,Nominee 2,Nominee 3,Nominee 4
0,Wings,The Racket,Seventh Heaven,Unique and Artistic Production- Sunrise: A Son...,Chang: A Drama of the Wilderness
1,The Broadway Melody,Alibi,Hollywood Revue,In Old Arizona,The Patriot
2,All Quiet on the Western Front,The Big House,Disraeli,The Divorcee,The Love Parade
3,Cimarron,East Lynne,The Front Page,Skippy,Trader Horn
4,Grand Hotel,Arrowsmith,Bad Girl,The Champ,Five Star Final
...,...,...,...,...,...
91,Parasite,1917,Ford v Ferrari,The Irishman,Jojo Rabbit
92,Nomadland,The Father,Judas and the Black Messiah,Mank,Minari
93,CODA,Belfast,Don't Look Up,Drive My Car,Dune
94,Everything Everywhere All At Once,All Quiet on the Western Front,Avatar: The Way of Water,The Banshees of Inisherin,Elvis
